<a href="https://colab.research.google.com/github/inokchoi/ComputerVision/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.datasets.cifar10 import load_data
import tensorflow as tf
import os
import numpy as np

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# (X_train, y_train), (X_test, y_test) = load_data()

mnist = input_data.read_data_sets("MNIST_data/", reshape=False, one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_valid, y_valid = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_valid)))
print("Test Set:       {} samples".format(len(X_test)))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Image Shape: (28, 28, 1)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


In [ ]:
def conv2d(input, filters, kernel_size, strides=1, activation=tf.nn.relu, padding='SAME', name='conv'):
    with tf.variable_scope(name, reuse=False):
        out = tf.layers.conv2d(input, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                         activation=activation, name=name)
    return out

def dense(input, unit, activation=tf.nn.relu, name='dense'):
    with tf.variable_scope(name, reuse=False):
        out = tf.layers.dense(input, unit, activation=activation, name=name)
    return out

def max_pool(input, k, s, name='pool'):
    out = tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='SAME', name=name)
    return out

def lrn(name, inputs, depth_radius=5, alpha=0.0001, beta=0.75):
    with tf.variable_scope(name, reuse=False):
        out = tf.nn.local_response_normalization(name=name, input=inputs, depth_radius=depth_radius, alpha=alpha, beta=beta)
    return out

In [ ]:
def inception(x, conv11_size, conv33_rsize, conv33_size, conv55_rsize, conv55_size, pool_size, name='incept'):
    conv1_1 = conv2d(x, conv11_size, [1,1], [1,1], name='{}_1x1'.format(name))

    conv3_3_red = conv2d(x, conv33_rsize, [1,1], [1,1], name='{}_3x3_red'.format(name))
    conv3_3 = conv2d(conv3_3_red, conv33_size, [3,3], [1,1], name='{}_3x3'.format(name))

    conv5_5_red = conv2d(x, conv55_rsize, [1,1], [1,1], name='{}_5x5_red'.format(name))
    conv5_5 = conv2d(conv5_5_red, conv55_size, [5,5], [1,1], name='{}_5x5'.format(name))

    pool = max_pool(x, 3, 1, name='{}_pool'.format(name))
    pool_conv = conv2d(pool, pool_size, [1,1],[1,1], name='{}_pool_conv'.format(name))

    out = tf.concat([conv1_1, conv3_3, conv5_5, pool_conv], axis=-1)

    return out

def googlenet(x):
    conv1 = conv2d(x, 64, [7,7], [2,2], name='conv1')
    pool1 = max_pool(conv1, 3, 2, name='pool1')
    pool1_norm1 = lrn('pool1_norm1', pool1)

    conv2 = conv2d(pool1_norm1, 64, [1,1], [1,1], name='conv2')
    conv3 = conv2d(conv2, 192, [3,3], [1,1], name='conv3')
    conv3_norm1 = lrn('conv3_norm1', conv3)
    pool2 = max_pool(conv3_norm1, 3, 2, name='pool1')

    inception1 = inception(pool2, 64,96,128,16,32,32, name='inception1')
    inception2 = inception(inception1, 128,128,192,32,96,64, name='inception2')
    pool3 = max_pool(inception2, 3, 2, name='pool3')

    inception3 = inception(pool3, 192,96,208,16,48,64, name='inception3')
    inception4 = inception(inception3, 160,112,224,24,64,64, name='inception4')
    inception5 = inception(inception4, 128,128,256,24,64,64, name='inception5')
    inception6 = inception(inception5, 112,144,288,32,64,64, name='inception6')
    inception7 = inception(inception6, 256,160,320,32,128,128, name='inception7')
    pool4 = max_pool(inception7, 3, 2, name='pool4')
    
    inception8 = inception(pool4, 256,160,320,32,128,128, name='inception8')
    inception9 = inception(inception8, 384,192,384,48,128,128, name='inception9')

    avg_pool1 = tf.nn.avg_pool(inception9, ksize=[1, 7, 7, 1], strides=[1, 1, 1, 1], padding='SAME', name='avg_pool1')
    dropout1 = tf.nn.dropout(avg_pool1,0.6)
    
    dense1 = tf.reshape(dropout1, [-1, tf.size(inception9[0])])
    dense2 = dense(dense1, 10, name='dense2')

    return dense2

In [ ]:
epochs = 100
learning_rate = 0.00001
batch_size = 64
dropout = 0.8


# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.int32, [None, 10])
# y_one_hot = tf.squeeze(tf.one_hot(y, 10), axis=1)

keep_prob = tf.placeholder(tf.float32, [])  # dropout (keep probability)
x_resize = tf.image.resize_images(x, [64, 64])

# Construct model
pred = googlenet(x)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,
                                      gpu_options=tf.GPUOptions(allow_growth=True, visible_device_list='0'))) as sess:
    sess.run(init)
    # Keep training until reach max iterations
    print('Training...')
    for i in range(epochs):
        s = np.random.permutation(len(X_train))
        X_train = X_train[s]
        y_train = y_train[s]
        loss_total, acc_total = 0, 0
        for offset in range(0, len(X_train), batch_size):
            end = offset + batch_size
            batch_xs, batch_ys = X_train[offset:end], y_train[offset:end]
            # Fit training using batch data
            _, acc, loss = sess.run([optimizer, accuracy, cost], feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})

            # Calculate batch accuracy
            # Calculate batch loss
            acc_total += acc
            loss_total += loss
        acc_total /= (len(X_train) / batch_size)
        loss_total /= (len(X_train) / batch_size)
        print("epoch: " + str(i) + ", Training Loss= " + "{:.4f}".format(
            loss_total) + ", Training Accuracy= " + "{:.4f}".format(acc_total))

    print("Optimization Finished!")
    saver.save(sess, './goolgenet')


Training...
epoch: 0, Training Loss= 1.5045, Training Accuracy= 0.4972
epoch: 1, Training Loss= 0.7099, Training Accuracy= 0.7543
epoch: 2, Training Loss= 0.5530, Training Accuracy= 0.8187
epoch: 3, Training Loss= 0.4627, Training Accuracy= 0.8572
epoch: 4, Training Loss= 0.3979, Training Accuracy= 0.8807
epoch: 5, Training Loss= 0.3474, Training Accuracy= 0.8982
epoch: 6, Training Loss= 0.3092, Training Accuracy= 0.9101
epoch: 7, Training Loss= 0.2829, Training Accuracy= 0.9199
epoch: 8, Training Loss= 0.2535, Training Accuracy= 0.9283
epoch: 9, Training Loss= 0.2288, Training Accuracy= 0.9364
epoch: 10, Training Loss= 0.2042, Training Accuracy= 0.9429
epoch: 11, Training Loss= 0.1875, Training Accuracy= 0.9480
epoch: 12, Training Loss= 0.1801, Training Accuracy= 0.9488
epoch: 13, Training Loss= 0.1625, Training Accuracy= 0.9543
epoch: 14, Training Loss= 0.1507, Training Accuracy= 0.9575
epoch: 15, Training Loss= 0.1445, Training Accuracy= 0.9595
epoch: 16, Training Loss= 0.1340, Trai